In [12]:
import numpy as np
from numba import cuda
import matplotlib.pyplot as plt

#### RGB to HSV
Convert RGB image to HSV color space<br>
rgb_image: input RGB image (height, width, 3)<br>
h_out: output Hue channel (height, width) - ranges from 0 to 360<br>
s_out: output Saturation channel (height, width) - ranges from 0 to 1<br>
v_out: output Value channel (height, width) - ranges from 0 to 1<br>
<br>
<b>Hue, Saturation, Value</b><br>
• H ∈[0..360] : "The color"<br>
0° or 360° = Red<br>
60° = Yellow<br>
120° = Green<br>
180° = Cyan<br>
240° = Blue<br>
300° = Magenta<br>
• S ∈[0..1]: “The colorfulness”<br>
0.0 = Gray (no color)<br>
1.0 = Full color intensity<br>
• V ∈[0..1]: “The brightness”<br>
0.0 = Black<br>
1.0 = Full brightness<br>

In [13]:
import numpy as np
from numba import cuda
import matplotlib.pyplot as plt

@cuda.jit
def RGB2HSV(rgb_in, h_out, s_out, v_out):
    tidx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    tidy = cuda.threadIdx.y + cuda.blockIdx.y * cuda.blockDim.y

    if tidx < rgb_in.shape[1] and tidy < rgb_in.shape[0]:
        # Preparation
        ## Scale R, G, B to [0..255] to [0..1]
        r = rgb_in[tidy, tidx, 0] / 255.0
        g = rgb_in[tidy, tidx, 1] / 255.0
        b = rgb_in[tidy, tidx, 2] / 255.0
        ## Find max and min among R, G, B ∈[0..1]
        cmax = max(r, max(g, b))
        cmin = min(r, min(g, b))
        ## ∆ = max−min
        diff = cmax - cmin
        
        # Calculate Hue
        if diff == 0: h = 0
        if cmax == r: h = (60 * ((g - b) / diff) % 6) 
        if cmax == g: h = (60 * ((b - r) / diff) + 2)
        if cmax == b: h = (60 * ((r - g) / diff) + 4)
        # Calculate Saturation
        s = 0 if cmax == 0 else (diff / cmax)
        # Calculate Value
        v = cmax
        
        # Scatter the results to separate arrays
        h_out[tidy, tidx] = h
        s_out[tidy, tidx] = s
        v_out[tidy, tidx] = v


def convert_rgb_to_hsv(rgb_image):
    # Create output arrays
    height= rgb_image.shape[0]
    width = rgb_image.shape[1]
    h_out = np.zeros((height, width), dtype=np.float32)
    s_out = np.zeros((height, width), dtype=np.float32)
    v_out = np.zeros((height, width), dtype=np.float32)
    
    # CPU to GPU
    d_rgb = cuda.to_device(rgb_image)
    d_h = cuda.to_device(h_out)
    d_s = cuda.to_device(s_out)
    d_v = cuda.to_device(v_out)
    
    # CPU set up grid and block dimensions
    block_dim = (16, 16)
    grid_dim = (
        (width + block_dim[0] - 1) // block_dim[0],
        (height + block_dim[1] - 1) // block_dim[1]
    )
    
    # GPU launch kernel
    RGB2HSV[grid_dim, block_dim](d_rgb, d_h, d_s, d_v)
    
    # GPU to CPU
    h_out = d_h.copy_to_host()
    s_out = d_s.copy_to_host()
    v_out = d_v.copy_to_host()
    
    return h_out, s_out, v_out

# Usage
image = plt.imread("image1.jpg")
h, s, v =convert_rgb_to_hsv(image)
print(f"Hue:", h)
print(f"Saturation:", s)
print(f"Value:", v)

Hue: [[5.3114753 5.3114753 5.3114753 ... 4.4745765 1.1612903 2.6666667]
 [5.3114753 5.3114753 5.3114753 ... 4.4745765 1.1612903 2.6666667]
 [5.3114753 5.3114753 5.3114753 ... 4.4745765 1.1612903 2.6666667]
 ...
 [1.5789474 1.5789474 1.5789474 ... 0.        0.        0.       ]
 [4.5       4.5       4.5       ... 4.        4.        4.       ]
 [4.5       4.5       4.5       ... 4.        4.        4.       ]]
Saturation: [[0.7176471  0.7218935  0.7305389  ... 0.5412844  0.54385966 0.5478261 ]
 [0.7176471  0.7218935  0.7305389  ... 0.5412844  0.54385966 0.5478261 ]
 [0.7218935  0.72619045 0.7305389  ... 0.5412844  0.54385966 0.5478261 ]
 ...
 [0.1948718  0.19791667 0.19895288 ... 0.05970149 0.05970149 0.05970149]
 [0.2        0.20304568 0.20304568 ... 0.07389162 0.07389162 0.07389162]
 [0.19323671 0.19607843 0.19607843 ... 0.07352941 0.07352941 0.07389162]]
Value: [[0.6666667  0.6627451  0.654902   ... 0.42745098 0.44705883 0.4509804 ]
 [0.6666667  0.6627451  0.654902   ... 0.42745098 0

#### HSV to RGB

In [14]:
@cuda.jit
def HSV2RGB(h_in, s_in, v_in, rgb_out):
    tidx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    tidy = cuda.threadIdx.y + cuda.blockIdx.y * cuda.blockDim.y
    
    if tidx < rgb_out.shape[1] and tidy < rgb_out.shape[0]:
        h = h_in[tidy, tidx]
        s = s_in[tidy, tidx]
        v = v_in[tidy, tidx]
        
        # Preparation
        d = h / 60.0
        h_i = int(d) % 6
        f = d - int(d)
        l = v * (1.0 - s)
        m = v * (1.0 - f * s)
        n = v * (1.0 - (1.0 - f) * s)
        
        # Conversion
        r, g, b = 0.0, 0.0, 0.0
        if 0 <= h < 60: r, g, b = v, n, l
        elif 60 <= h < 120: r, g, b = m, v, l
        elif 120 <= h < 180: r, g, b = l, v, n
        elif 180 <= h < 240: r, g, b = l, m, v
        elif 240 <= h < 300: r, g, b = n, l, v
        elif 300 <= h < 360: r, g, b = v, l, m
            
        # Scale to 0-255 range and store in output
        rgb_out[tidy, tidx, 0] = int(r * 255)
        rgb_out[tidy, tidx, 1] = int(g * 255)
        rgb_out[tidy, tidx, 2] = int(b * 255)

def convert_hsv_to_rgb(h, s, v):
    # Create output array
    height= h.shape[0]
    width = h.shape[1]
    rgb_out = np.zeros((height, width, 3), dtype=np.uint8)
    
    # CPU to GPU
    d_h = cuda.to_device(h)
    d_s = cuda.to_device(s)
    d_v = cuda.to_device(v)
    d_rgb = cuda.to_device(rgb_out)
    
    # CPU ask GPU
    block_dim = (16, 16)
    grid_dim = (
        (width + block_dim[0] - 1) // block_dim[0],
        (height + block_dim[1] - 1) // block_dim[1]
    )
    
    # GPU launch kernel
    HSV2RGB[grid_dim, block_dim](d_h, d_s, d_v, d_rgb)
    
    # GPU to CPU
    rgb_out = d_rgb.copy_to_host()
    return rgb_out


# Usage
reconstructed_rgb = convert_hsv_to_rgb(h,s,v)
print(reconstructed_rgb)

[[[170  58  47]
  [169  57  47]
  [167  55  45]
  ...
  [109  54  50]
  [114  53  51]
  [115  54  51]]

 [[170  58  47]
  [169  57  47]
  [167  55  45]
  ...
  [109  54  50]
  [114  53  51]
  [115  54  51]]

 [[169  57  47]
  [168  56  46]
  [167  55  45]
  ...
  [109  54  50]
  [114  53  51]
  [115  54  51]]

 ...

 [[195 158 157]
  [192 155 154]
  [191 154 153]
  ...
  [201 189 189]
  [201 189 189]
  [201 189 189]]

 [[200 163 160]
  [197 160 157]
  [197 160 157]
  ...
  [203 189 188]
  [203 189 188]
  [203 189 188]]

 [[207 170 167]
  [204 167 164]
  [204 167 164]
  ...
  [204 190 189]
  [204 190 189]
  [203 189 188]]]
